# Object detection using Amazon Rekognition

***
This notebook provides a walkthrough of [object detection API](https://docs.aws.amazon.com/rekognition/latest/dg/labels.html) in Amazon Rekognition to identify objects.
***

## Initialization

In [ ]:
# Initialise Notebook
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os
import io

In [ ]:
# Curent AWS Region. Use this to choose corresponding S3 bucket with sample content

region = boto3.session.Session().region_name

In [ ]:
# Init clients
rekognition = boto3.client('rekognition')

In [ ]:
# Create temporary directory
# This directory is not needed to call Rekognition APIs.
# We will only use this directory to download images from S3 bucket and draw bounding boxes

!mkdir tmpfolder
temp_folder = 'tmpfolder/'

## Detect objects in image
***

### Option 1: Upload your own image

Upload any `.jpg` or `.png` image that you want to use for this Demo and assign the file path to the variable `image_path`.

You can upload through the button on the top left corner:

<img src="./img/img_upload.jpg">

### Option 2: Use the sample photo on the media/ folder

In [ ]:
image_path = "media/object-detection/skateboard.jpg"

In [ ]:
display(Image.open(image_path))

#### Call Rekognition to detect objects in the image

In [ ]:
# call Rekognition API of DetectLabels
with open(image_path, 'rb') as image:
    detect_label_response = rekognition.detect_labels(Image={'Bytes': image.read()})

## Review the raw JSON reponse from Rekognition

In [ ]:
# Show JSON response returned by Rekognition Labels API (Object Detection)
# In the JSON response below, you will see Label, detected instances, confidence score and additional information.

display(detect_label_response)

In [ ]:
# Define a function that will display image with bounded boxes around recognized objects
# We will call this function in next step
  
def draw_bounding_boxes(image_path, boxes):
    """
    Helper Function to draw bounding boxes around detected objects
    input: image path 
    returns: displayble images with detected bounding boxes
    """
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    width, height = img.size
    line = 3
    for box in boxes:
        x1 = int(box[1]['Left'] * width)
        y1 = int(box[1]['Top'] * height)
        x2 = int(box[1]['Left'] * width + box[1]['Width'] * width)
        y2 = int(box[1]['Top'] * height + box[1]['Height']  * height)
        fnt_size = 20
        fnt = ImageFont.truetype("/usr/share/fonts/dejavu/DejaVuSans.ttf", fnt_size)
        draw.text((x1,y1),box[0],font=fnt)
        for l in range(line):
            draw.rectangle((x1-l,y1-l,x2+l,y2+l))
        
    display(img)

In [ ]:
# Show image and bounded boxes around detected objects
boxes = []
objects = detect_label_response['Labels']
for obj in objects:
    for instance in obj["Instances"]:
        boxes.append((obj['Name'], instance['BoundingBox']))
    
draw_bounding_boxes(image_path, boxes)

## Worker Safety with Amazon Rekognition
***
You can use Amazon Rekognition to detect if certain objects are not present in the image or video. For example you can perform worker safety audit by revieweing images/video of a construction site  and detecting if there are any workers without safety hat.

In [ ]:
image_path = "media/ppe-detection/ppe_group.jpg"

In [ ]:
display(Image.open(image_path))

In [ ]:
# Call Amazon Rekognition to detect PPE in the image
# https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectProtectiveEquipment.html
with open(image_path, 'rb') as image:
    detect_ppe_response = rekognition.detect_protective_equipment(Image={'Bytes': image.read()})

In [ ]:
def detect_ppe(image_path, detect_ppe_response, confidence):

    fill_green='#00d400'
    fill_red='#ff0000'
    fill_yellow='#ffff00'
    line_width=3
    
    # open image and get image data from stream.
    image = Image.open(open(image_path,'rb'))
    stream = io.BytesIO()
    image.save(stream, format=image.format)    
    image_binary = stream.getvalue()
    imgWidth, imgHeight = image.size  
    draw = ImageDraw.Draw(image)  
    
    
    for person in detect_ppe_response['Persons']:
        found_mask = False
        found_hat = False 
        for body_part in person['BodyParts']:
            ppe_items = body_part['EquipmentDetections']
                 
            for ppe_item in ppe_items:
                if ppe_item['Type'] == 'FACE_COVER':
                    fill_color=fill_green
                    found_mask=True
                    # check if mask covers face
                    if ppe_item['CoversBodyPart']['Value'] == False:
                        fill_color=fill='#ff0000'
                    # draw bounding box around mask
                    box = ppe_item['BoundingBox']
                    left = imgWidth * box['Left']
                    top = imgHeight * box['Top']
                    width = imgWidth * box['Width']
                    height = imgHeight * box['Height']
                    points = (
                            (left,top),
                            (left + width, top),
                            (left + width, top + height),
                            (left , top + height),
                            (left, top)
                        )
                    draw.line(points, fill=fill_color, width=line_width)
                    
                     # Check if confidence is lower than supplied value       
                    if ppe_item['CoversBodyPart']['Confidence'] < confidence:
                        # draw warning yellow bounding box within face mask bounding box
                        offset=line_width+ line_width 
                        points = (
                                    (left+offset,top + offset),
                                    (left + width-offset, top+offset),
                                    ((left) + (width-offset), (top-offset) + (height)),
                                    (left+ offset , (top) + (height -offset)),
                                    (left + offset, top + offset)
                                )
                        draw.line(points, fill=fill_yellow, width=line_width)
                               
                    
                if ppe_item['Type'] == 'HEAD_COVER': # you can change it to "HEAD_COVER" or "HAND_COVER"
                    fill_color = '#00d460'
                    found_hat = True

                    if ppe_item['CoversBodyPart']['Value'] == False:
                        fill_color=fill='#ff0000'
                        
                    # draw bounding box around hat
                    box = ppe_item['BoundingBox']
                    left = imgWidth * box['Left']
                    top = imgHeight * box['Top']
                    width = imgWidth * box['Width']
                    height = imgHeight * box['Height']
                    points = (
                            (left,top),
                            (left + width, top),
                            (left + width, top + height),
                            (left , top + height),
                            (left, top)
                        )
                    draw.line(points, fill=fill_color, width=line_width)
                    # Check if confidence is lower than supplied value       
                    if ppe_item['CoversBodyPart']['Confidence'] < confidence:
                        # draw warning yellow bounding box within the bounding box
                        offset=line_width+ line_width 
                        points = (
                                    (left+offset,top + offset),
                                    (left + width-offset, top+offset),
                                    ((left) + (width-offset), (top-offset) + (height)),
                                    (left+ offset , (top) + (height -offset)),
                                    (left + offset, top + offset)
                                )
                        draw.line(points, fill=fill_yellow, width=line_width)

                    
        if found_mask==False or found_hat==False:
            box = person['BoundingBox']
            left = imgWidth * box['Left']
            top = imgHeight * box['Top']
            width = imgWidth * box['Width']
            height = imgHeight * box['Height']
            points = (
                (left,top), (left + width, top),
                (left + width, top + height),
                (left , top + height), (left, top))
            draw.line(points, fill=fill_red, width=line_width)

    display(image)

In [ ]:
confidence = 90 # confidence if the bodypart is covered by the particular PPE 
detect_ppe(image_path, detect_ppe_response, confidence)

### Learn more about building an end to end worker safety app
To learn on how to build an end to end worker safety app, learn more at https://github.com/aws-samples/aws-deeplens-worker-safety-project